導入Lib

In [13]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split, RandomSampler
import time
import os
# os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

# 1. 準備數據集

In [14]:
# 下載數據
train_data = torchvision.datasets.CIFAR10(root="./dataset", train=True, download=True, transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.CIFAR10(root="./dataset", train=False, download=True, transform=torchvision.transforms.ToTensor())

train_data_size = len(train_data)
test_data_size = len(test_data)
print(f"訓練數據集的長度為:{train_data_size}")
print(type(train_data))
print(f"測試數據集的長度為:{test_data_size}")
print(type(test_data))

# 用DataLoader加載數據集  # 打亂數據
train_loader = DataLoader(train_data, batch_size=64, sampler=RandomSampler(train_data, replacement=False)) # replacement=False 一個樣本只采樣一次 # sampler定義取bitch的方法
test_loader = DataLoader(test_data, batch_size=64, shuffle=True) # shuffle=True 會打亂順序



Files already downloaded and verified
Files already downloaded and verified
訓練數據集的長度為:50000
<class 'torchvision.datasets.cifar.CIFAR10'>
測試數據集的長度為:10000
<class 'torchvision.datasets.cifar.CIFAR10'>


# 2. 塔建神經網絡

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(), # 激活層通常在 Conv或Linear 後, 除了最後一層
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(in_features=64 * 4 * 4, out_features=64), # out_channels是64 所以in_features是64, 32*32的圖片經過三次池化變成4*4
            nn.ReLU(),
            nn.Dropout(p=0.2), # 每個數據有20%概率歸零, Dropout通常在全連拉層後的激活層後
            nn.Linear(in_features=64, out_features=10)
        )

    def forward(self, x):
        x = self.model(x)
        return x


# 3. 訓練和測試模型
GPU方法用在 模型,損失函數,數據

In [18]:
# 定義訓練的設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 創造網絡模型

net = Net()
net.to(device)


# 損失數
criterion = nn.CrossEntropyLoss()
criterion.to(device)

# 優化器
# 1e-2 = 1*(10)^(-2) = 0.01
learning_rate = 0.003 # 學習速度
# optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# 設置訓練網絡的一些參數
# 記錄訓練的次數
total_train_step = 0
# 記錄測試的次數
total_test_step = 0
# 訓練的輪數
epoch = 30

# 添加tensorboard
writer = SummaryWriter("./template_logs")

# 開始計時
start_time = time.time()

for i in range(epoch):
    print(f"~~~~~~~~~~~~~~~~~~~~第{i + 1}輪訓練開始~~~~~~~~~~~~~~~~~~~~")

    #開始訓練
    net.train()
    for train_data in train_loader:
        train_imgs, train_labels = train_data
        train_imgs = (train_imgs.float() - train_imgs.float().mean()) / train_imgs.float().std()
        train_imgs = train_imgs.to(device)
        train_labels = train_labels.to(device)

        # 獲得10個分類的概率
        train_output = net(train_imgs)
        # 損失
        train_loss = criterion(train_output, train_labels)

        # 優化器優化模型
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        total_train_step += 1
        if total_train_step % 100 == 0:
            # print(f"訓練次數: {total_train_step}, Loss: {train_loss}")
            # train_loss的dtype是torch.Tensor     train_loss.item()的dtype是float
            # print(type(train_loss), type(train_loss.item()))
            writer.add_scalar("train_loss", train_loss, total_train_step)


    # 記錄誤差和
    total_test_loss = 0
    # 整體正確個數
    total_accuracy = 0
    # 測試步驟開始
    net.eval()
    with torch.no_grad(): # 裏面的代碼没有梯度, 保証不會對其進行調優
        for test_data in test_loader:
            test_imgs, test_labels = test_data
            test_imgs = test_imgs.to(device)
            test_labels = test_labels.to(device)

            test_output = net(test_imgs)

            test_loss = criterion(test_output, test_labels)
            total_test_loss += test_loss.item()

            accuracy = (test_output.argmax(dim=1) == test_labels).sum() # 預測正確個數
            total_accuracy += accuracy

    prob = total_accuracy / test_data_size * 100
    print(f"整體測試集上的Loss: {total_test_loss}")
    print(f"整體測試集上的正確率: {prob:.3f}%")
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy / test_data_size, total_test_step)
    total_test_step += 1

    model = f"net_{i+1}_{prob:.3f}.pth"
    torch.save(net.state_dict(), f"./models/complete_template/{model}")
    print(f"模型 {model} 已保存")

# 結束計時
end_time = time.time()
print(f"用时: {end_time - start_time}")
writer.close()


~~~~~~~~~~~~~~~~~~~~第1輪訓練開始~~~~~~~~~~~~~~~~~~~~
整體測試集上的Loss: 334.7808505296707
整體測試集上的正確率: 23.990%
模型 net_1_23.990.pth 已保存
~~~~~~~~~~~~~~~~~~~~第2輪訓練開始~~~~~~~~~~~~~~~~~~~~


KeyboardInterrupt: 

如果图片是png,要加img=image.convert('RGB')

因為png格式是四個通道,除了RGB三個通道,还有一个透明通道, 用上方代碼保留其顏色通道